### 1. Stress Event Model (Operational Risk)

**Question it answers**

> *“What is the probability that the pipeline is under operational stress?”*

This model focuses on **operational conditions**, not price behavior.

### What goes into the model
- Pipeline notices and alerts
- Indicators of operational disruption
- Weather-driven demand pressure (HDD)
- Storage tightness (supply buffer)

### What the model produces
- A probability between **0 and 1** representing the likelihood of operational stress

### Interpretation
- A high value means **conditions are abnormal**
- A low value means **operations appear normal**

This model acts as an **early-warning system** for potential pipeline issues.


In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [12]:
from models.features.build_model_frame import ModelFrameConfig, build_model_frame
from models.features.derive_forecast_inputs import derive_forecast_inputs
from models.stress.stress_event_model import fit_stress_event_model
from models.stress.forecast_stress_event import forecast_stress_event_prob

In [5]:
from data.mongo import load_henry_hub_daily, load_storage_weekly, \
                    load_noaa_region_daily, \
                    load_capacity_df, load_notices_df

In [6]:
MONGO_DB = os.getenv("MONGO_DB")
MONGO_URI = os.getenv("MONGO_URI")

### 1. Build Model Frame

In [7]:
cfg = ModelFrameConfig(
    pipeline="algonquin",
    capacity_collection="ebb_algonquin_capacity",
    notices_collection="ebb_algonquin_notices",
    start="2025-10-01",
)

In [8]:
df=build_model_frame(cfg, mongo_uri=MONGO_URI, mongo_db=MONGO_DB)

### 2. Fit Stress Event Model

In [9]:
model_event, idata_event, scalers_event = fit_stress_event_model(df)

 Progress                   Draws   Divergences   Step size   Grad evals   Sampling Speed    Elapsed   Remaining  
 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  ━━━━━━━━━━━━━━━━━━━━━━━━   3000    0             0.70        7            1249.48 draws/s   0:00:02   0:00:00    
  ━━━━━━━━━━━━━━━━━━━━━━━━   3000    0             0.95        3            1315.82 draws/s   0:00:02   0:00:00    
  ━━━━━━━━━━━━━━━━━━━━━━━━   3000    0             0.71        7            1200.87 draws/s   0:00:02   0:00:00    
  ━━━━━━━━━━━━━━━━━━━━━━━━   3000    0             0.79        7            798.54 draws/s    0:00:03   0:00:00

Sampling 4 chains for 1_500 tune and 1_500 draw iterations (6_000 + 6_000 draws total) took 17 seconds.


In [10]:
x = derive_forecast_inputs(df)

In [13]:
p_samp, p_alert = forecast_stress_event_prob(model_event, idata_event, x=x, scalers=scalers_event)



### 4. Out-of-Sample Checks

### 5. Diagnostics & Sanity Checks